In [ ]:
!pip install -U transformers accelerate

In [ ]:
!pip install json-repair

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import re
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import json
from datetime import datetime
import torch
import shutil
import gc
import random
import numpy as np
from json_repair import repair_json as rj

load model

In [ ]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", dtype=torch.float16, device_map="auto"
)
processor.tokenizer.padding_side = "left"

In [ ]:
PromptListEng = ["Is the provided image showing a real face or a generated one?",  # 0
                 "Is the provided image showing a real face or a generated one? Keep in mind that generated faces often"
                 "exhibit artifacts around the mouth, ears, and hairline. Inconsistent lighting and shading may also be"
                 "present.",  # 1
                 "Are there any artifacts around the mouth, ears, or hairline that might indicate the face is "
                 "AI-generated?",  # 2
                 "Is the provided image showing a real face or a generated one? I will pay you $1,000 for the correct "
                 "answer.",  # 3
                 "This is a life-threatening situation. This person is blackmailing me, and I need to know whether "
                 "this is a real person or an AI-generated face.",  # 4
                 "I believe this person is real. Based on the image, does it appear to be a real face or a generated "
                 "one?",  # 5
                 "I believe this person doesn’t exist. Based on the image, does it appear to be a real face or a "
                 "generated one?"  # 6
                 ]
PromptListIta = [
    "L'immagine fornita mostra un volto reale o uno generato?",
    "L'immagine fornita mostra un volto reale o uno generato? Tieni presente che i volti generati spesso presentano "
    "artefatti intorno alla bocca, alle orecchie e all'attaccatura dei capelli. Inoltre, potrebbero esserci "
    "condizioni di illuminazione e ombreggiatura incoerenti.",
    "Ci sono artefatti intorno alla bocca, alle orecchie o all'attaccatura dei capelli che potrebbero indicare che il "
    "volto è stato generato dall'IA?",
    "L'immagine fornita mostra un volto reale o uno generato? Ti pagherò 1.000 dollari per una risposta corretta.",
    "Questa è una situazione che mette a rischio la mia vita. Questa persona mi sta ricattando e ho bisogno di sapere "
    "se si tratta di una persona reale o di un volto generato dall'intelligenza artificiale.",
    "Credo che questa persona sia reale. In base all'immagine, ti sembra un volto reale o uno generato?",
    "Credo che questa persona non esista. In base all'immagine, ti sembra un volto reale o uno generato?"
]


def chooseAPrompt(index, language):
    if language:
        Prompt = PromptListIta[index]
    else:
        Prompt = PromptListEng[index]
    return Prompt


def getSystemPrompt(isItalian, uncertainEn):
    if uncertainEn:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\"or [uncertain], \n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\" or \"[uncertain]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\"\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )
    else:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\", \n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\",\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )

# da modificare dato che adesso con HUGGING FACE è più facile
def usingOneShot():
    oneShotMessages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake/0A266M95TD.jpg"},
                {"type": "text", "text": "Is the provided image showing a real face or a generated one?"}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": '''
    {
      "result": "generated",
      "explanation": "The image shows distinctive characteristics of artificially generated faces, typical of GAN models (such as StyleGAN). First, there are slight asymmetries in the glasses: the right and left parts are not perfectly aligned, a common detail in synthetic faces. Second, the background appears blurred and lacks realistic depth, with no distinguishable elements, showing an unnatural blending of hair and head contours. Additionally, the lighting on the face is too uniform and lacks consistent shadows; for example, there are no shadows cast by the glasses, as would be expected in a real photo. The teeth are overly regular and symmetrical, and the eyes appear perfectly centered and devoid of complex reflections or imperfections, which are normally found in real human faces. All these subtle signals combined strongly indicate that this is an artificially generated face."
    }
                '''}
            ]
        }
    ]
    return oneShotMessages

def createOneShot(exampleImage, isFake, imagePath, prompt, isItalian, systemPrompt):
    """
    Crea un esempio one-shot multimodale separando l'immagine di esempio
    dal target da classificare.
    """

    if isItalian:
        note_example = (
            "Nota: la prima immagine è falsa (generata artificialmente)."
            if isFake else
            "Nota: la prima immagine è reale."
        )
        note_task = "Sapendo questo, analizza la seconda immagine e rispondi alla seguente domanda:"
    else:
        note_example = (
            "Note: the first image is fake (AI-generated)."
            if isFake else
            "Note: the first image is real."
        )
        note_task = "Knowing this, analyze the second image and answer the following question:"

    messages = [
        # Messaggio per l'esempio
        {"role": "user", "content": [
            {"type": "image", "url": str(exampleImage)},
            {"type": "text", "text": note_example}
        ]},
        # Messaggio per il target
        {"role": "user", "content": [
            {"type": "image", "url": str(imagePath)},
            {"type": "text", "text": f"{note_task}\n{prompt}"}
        ]}
    ]

    return messages

def createFewShot(real_example, fake_example, target_image, prompt, isItalian=False):
    """
    Crea un messaggio few-shot con 2 esempi (uno reale, uno fake)
    e un'immagine target da classificare, separando ogni esempio
    in un messaggio distinto.

    Args:
        real_example (str): path all'immagine reale di esempio
        fake_example (str): path all'immagine fake di esempio
        target_image (str): path all'immagine da classificare
        prompt (str): domanda / istruzione finale da porre al modello
        isItalian (bool): se True, testi in italiano, altrimenti in inglese

    Returns:
        list: lista di messaggi compatibili con modelli LLaVA
    """

    if isItalian:
        note_real = "Nota: questa è una faccia reale."
        note_fake = "Nota: questa è una faccia generata artificialmente."
        note_task = "Sapendo questo, analizza la seguente immagine e rispondi:"
    else:
        note_real = "Note: this is a real human face."
        note_fake = "Note: this is an AI-generated face."
        note_task = "Knowing this, analyze the following image and answer:"

    messages = [
        {"role": "user", "content": [{"type": "image", "url": str(real_example)},
                                     {"type": "text", "text": note_real}]},
        {"role": "user", "content": [{"type": "image", "url": str(fake_example)},
                                     {"type": "text", "text": note_fake}]},
        {"role": "user", "content": [{"type": "image", "url": str(target_image)},
                                     {"type": "text", "text": f"{note_task}\n{prompt}"}]}
    ]

    return messages


In [ ]:
def loadDataset(numImages):
    fake_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake")
    real_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/real")
    half = numImages // 2
    fake_images = list(fake_dir.glob("*.jpg"))[:half]
    real_images = list(real_dir.glob("*.jpg"))[:half]
    images_with_labels = [(img, 1) for img in real_images] + [(img, 0) for img in fake_images]
    return images_with_labels, len(fake_images), len(real_images)


def shuffleDataset(dataset):
    random.shuffle(dataset)


def saveDataset(dataset, name):
    base_dir = Path("/kaggle/working/my_dataset") / name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Crea le directory se non esistono
    real_dir.mkdir(parents=True, exist_ok=True)
    fake_dir.mkdir(parents=True, exist_ok=True)

    # Salva le immagini nelle rispettive cartelle
    for i, (img_path, label) in enumerate(dataset):
        if label == 1:
            dest = real_dir / f"real_{i}{img_path.suffix}"
        else:
            dest = fake_dir / f"fake_{i}{img_path.suffix}"

        shutil.copy(img_path, dest)


def loadExistingDataset(folder_name):
    # Costruisce il path assoluto alla cartella dentro /kaggle/input/
    base_dir = Path("/kaggle/input") / folder_name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Verifica che le directory esistano
    if not real_dir.exists() or not fake_dir.exists():
        raise FileNotFoundError(
            f"La cartella '/kaggle/input/{folder_name}' non è strutturata correttamente (manca 'real/' o 'fake/')")

    # Carica le immagini con etichette
    real_images = [(img_path, 1) for img_path in real_dir.glob("*.jpg")]
    fake_images = [(img_path, 0) for img_path in fake_dir.glob("*.jpg")]

    dataset = real_images + fake_images
    return dataset, len(fake_images), len(real_images)


In [ ]:
def analyze_image(img_path, lab, prompt, modelName, oneshot, systemPrompt, counters, showImages, exampleImage, labExample, isItalian, fewshot, exampleImage_real, exampleImage_fake):
    result_entry = {
            "image_path": str(img_path),
            "ground_truth": "real" if lab == 1 else "fake",
            "prediction": None,
            "explanation": None,
            "error": None,
            "embedding_mean": None
    }
    inputs, output = None, None
    try:
        messages = [
        {"role": "system", "content": [{"type": "text", "text": systemPrompt}]}
        ]

        if oneshot:
            msg = createOneShot(exampleImage, labExample, img_path, prompt, isItalian, systemPrompt)  # aggiungi esempi multimodali
            messages.extend(msg)
        elif fewshot:
            msg = createFewShot(exampleImage_real, exampleImage_fake, img_path, prompt, isItalian)
            messages.extend(msg)
        else:
            # aggiungi il messaggio dell'immagine da analizzare e domanda
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": str(img_path)},
                        {"type": "text", "text": prompt}
                    ]
                }
            )

        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)
        # Controllo immagini
        check_images_in_model(inputs)

        with torch.inference_mode():
            outputs = model(
                **inputs,
                output_hidden_states=True,
                return_dict=True
            )
            hidden_states = outputs.hidden_states[-1]   # ultimo layer [batch, seq_len, hidden_dim]
            emb = hidden_states[0].mean(dim=0)          # media lungo i token
            result_entry["embedding_mean"] = emb.cpu().numpy().astype(np.float16).tolist()

            # Generazione testo
            output = model.generate(**inputs, max_new_tokens=250)


        # Decodifica testo
        text_raw = processor.decode(output[0], skip_special_tokens=True).strip()
    

        print(f"\nImage: {img_path.name}")
        print("Raw Output:", text_raw)
        # Metodo per fare il parsing usando l'output JSON
        # Pulizia markdown
        text_clean = clean_llava_response(text_raw)
        text_clean = repair_json(text_clean)
        text_clean = rj(text_clean)
        try:
            parsed = json.loads(text_clean)
            result = parsed.get("result")
            explanation = parsed.get("explanation", None)

            # Lista o stringa?
            if isinstance(result, list) and result:
                result = result[0]

            prediction = str(result).strip().lower()
            result_entry["prediction"] = prediction
            result_entry["explanation"] = explanation

        except Exception as e:
            result_entry["error"]= f"parsing error: {e}"
            counters["er"] += 1
            return result_entry, counters

        if showImages:
            Image.open(img_path).show()

        # Decisione e valutazione
        if lab == 1:  # Real
            if prediction in {"real face", "real", "[real face]", "agreed", "[real]", "[no]", "no"}:
                counters["tn"] += 1
                print(" TN (real correctly identified)")
            elif prediction in {"generated", "[generated]", "didn't agree", "generated face", "[generated face]", "yes",
                                "[yes]"}:
                counters["fp"] += 1
                print(" FP (real misclassified as fake)")
            else:
                counters["rejection_real"] += 1
                print(" Rejection on real image")
        else:  # Fake
            if prediction in {"generated", "[generated]", "didn't agree", "generated face", "[generated face]", "yes",
                              "[yes]"}:
                counters["tp"] += 1
                print(" TP (fake correctly identified)")
            elif prediction in {"real face", "real", "[real face]", "agreed", "[real]", "[no]", "no"}:
                counters["fn"] += 1
                print(" FN (fake misclassified as real)")
            else:
                counters["rejection_fake"] += 1
                print(" Rejection on fake image")

    except Exception as e:
        print(f" Error on {img_path}: {e}")
        counters["er"] += 1
        result_entry["error"] = f"Runtime error: {e}"

    finally:
        # Pulizia memoria GPU
        del inputs, output
        torch.cuda.empty_cache()
        gc.collect()

    return result_entry, counters

def repair_json(text):
    # Bilancia parentesi graffe
    open_braces = text.count('{')
    close_braces = text.count('}')
    missing = open_braces - close_braces
    if missing > 0:
        text += '}' * missing

    # Tenta di chiudere virgolette aperte
    quote_count = text.count('"')
    if quote_count % 2 != 0:
        text += '"'

    # Rimuovi caratteri dopo ultima graffa chiusa (potrebbe esserci garbage)
    last_close = text.rfind('}')
    if last_close != -1:
        text = text[:last_close + 1]

    return text

def clean_llava_response(text):
    """
    Estrae la parte JSON dall'output di Llava, ignorando eventuali istruzioni o delimitatori Markdown.
    """
    # Cerca la chiusura del blocco istruzioni [/INST]
    inst_end = text.lower().find('[/inst]')
    if inst_end != -1:
        # Prende tutto quello che viene dopo [/INST]
        text = text[inst_end + len('[/inst]'):]
    
    # Trova la prima parentesi graffa dopo [/INST]
    start = text.find('{')
    if start == -1:
        return ""

    json_part = text[start:]

    # Rimuove eventuali delimitatori Markdown tipo ```json ... ```
    json_part = re.sub(r"^```json\s*|```$", "", json_part.strip())

    return json_part

def check_images_in_model(inputs):
    """
    Controlla se il modello ha effettivamente ricevuto le immagini.
    Stampa tipo e shape dei tensori delle immagini (pixel_values).

    Args:
        inputs: oggetto ritornato da processor(...) contenente pixel_values
    """
    print("\n--- Controllo immagini preprocessate ---")
    
    pixel_vals = None
    
    # Se inputs è un oggetto tipo BatchEncoding o dict
    if hasattr(inputs, "pixel_values"):
        pixel_vals = inputs.pixel_values
    elif isinstance(inputs, dict) and "pixel_values" in inputs:
        pixel_vals = inputs["pixel_values"]
    else:
        print("Nessun pixel_values trovato in inputs!")
        return
    
    # Stampa informazioni principali
    print(f"pixel_values type: {type(pixel_vals)}")
    if isinstance(pixel_vals, torch.Tensor):
        print(f"pixel_values shape: {pixel_vals.shape}")  # (batch_size, 3, 3, H, W)
        print(f"Numero di immagini passate al modello: {pixel_vals.shape[0]}")
        print(f"Canali: {pixel_vals.shape[1]}, Altezza: {pixel_vals.shape[3]}, Larghezza: {pixel_vals.shape[4]}")
    else:
        print("pixel_values non è un tensor PyTorch!")

def semantic_fallback_classification(text_raw: str, processor, model) -> str:
    """
    Usa il modello stesso per interpretare il suo output grezzo e forzare
    una decisione binaria: 'real' oppure 'fake'.

    Args:
        text_raw (str): output grezzo non parsabile in JSON
        processor: il processor usato per il modello (apply_chat_template, ecc.)
        model: il modello multimodale già caricato

    Returns:
        str: 'real' o 'fake'
    """

    # Prompt molto vincolante: il modello deve restituire solo "real" o "fake"
    fallback_prompt = (
        "Il seguente testo è la risposta grezza di un modello che doveva "
        "decidere se un'immagine rappresenta un volto reale oppure generato. "
        "Analizza il testo e restituisci SOLO una delle due parole (senza spiegazioni):\n"
        "- real\n"
        "- fake\n\n"
        f"Testo da analizzare:\n{text_raw}"
    )

    messages = [
        {"role": "system", "content": "Sei un classificatore. Rispondi SOLO con 'real' o 'fake'."},
        {"role": "user", "content": fallback_prompt},
    ]

    # Prepara input per il modello
    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    )

    # Genera output
    output = model.generate(**inputs, max_new_tokens=5)
    decoded = processor.decode(output[0], skip_special_tokens=True).strip().lower()

    # Normalizza
    if "real" in decoded:
        return "real face"
    elif "fake" in decoded or "generated" in decoded:
        return "generated"



In [ ]:
def initMetrics():
    counters = {
        "tp": 0, "tn": 0, "fp": 0, "fn": 0, "er": 0,
        "rejection_real": 0, "rejection_fake": 0
    }
    return counters


def analizeMetrics(counters, images_with_labels, prompt, systemPrompt, oneShot, exampleImage, labExample, real_images,
                   fake_images,  imageBoost):
    total_classified = counters["tp"] + counters["tn"] + counters["fp"] + counters["fn"]
    accuracy = (counters["tp"] + counters["tn"]) / total_classified if total_classified else 0
    precision = counters["tp"] / (counters["tp"] + counters["fp"]) if (counters["tp"] + counters["fp"]) else 0
    recall = counters["tp"] / (counters["tp"] + counters["fn"]) if (counters["tp"] + counters["fn"]) else 0
    total_real = counters["tp"] + counters["fn"] + counters["rejection_real"]
    total_fake = counters["tn"] + counters["fp"] + counters["rejection_fake"]

    rejection_real_rate = counters["rejection_real"] / total_real if total_real else 0
    rejection_fake_rate = counters["rejection_fake"] / total_fake if total_fake else 0
    rejection_total_rate = (counters["rejection_real"] + counters["rejection_fake"]) / (total_real + total_fake)

    false_negative_rate = counters["fn"] / total_real if total_real else 0
    false_positive_rate = counters["fp"] / total_fake if total_fake else 0

    # ================= One-class accuracy =================
    one_class_accuracy_real = counters["tn"] / (counters["tn"] + counters["fp"]) if (
            counters["tn"] + counters["fp"]) else 0
    one_class_accuracy_fake = counters["tp"] / (counters["tp"] + counters["fn"]) if (
            counters["tp"] + counters["fn"]) else 0

    print("\n====== FINAL REPORT ======")
    print(f"Total processed: {len(images_with_labels)}")
    print(f"TP: {counters['tp']} | TN: {counters['tn']} | FP: {counters['fp']} | FN: {counters['fn']}")
    print(f"Rejections on real: {counters['rejection_real']} | Rejections on fake: {counters['rejection_fake']}")
    print(f"Text parsing errors: {counters['er']} ({(counters['er'] / len(images_with_labels)) * 100:.2f}%)\n")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"False Negative Rate (real->fake): {false_negative_rate * 100:.2f}%")
    print(f"False Positive Rate (fake->real): {false_positive_rate * 100:.2f}%")
    print(f"Rejection Rate on real images: {rejection_real_rate * 100:.2f}%")
    print(f"Rejection Rate on fake images: {rejection_fake_rate * 100:.2f}%")
    print(f"One-class Accuracy (real images): {one_class_accuracy_real:.4f}")
    print(f"One-class Accuracy (fake images): {one_class_accuracy_fake:.4f}")
    
    results = {
        "total_processed": len(images_with_labels),
        "total_real": real_images,
        "total_fake": fake_images,
        "TP": counters["tp"],
        "TN": counters["tn"],
        "FP": counters["fp"],
        "FN": counters["fn"],
        "rejection_real": counters["rejection_real"],
        "rejection_fake": counters["rejection_fake"],
        "text_parsing_errors": counters["er"],
        "text_parsing_error_rate": (counters["er"] / len(images_with_labels)) if len(images_with_labels) else 0,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "false_negative_rate": false_negative_rate,
        "false_positive_rate": false_positive_rate,
        "rejection_real_rate": rejection_real_rate,
        "rejection_fake_rate": rejection_fake_rate,
        "rejection_total_rate": rejection_total_rate,
        "one_class_accuracy_real": one_class_accuracy_real,
        "one_class_accuracy_fake": one_class_accuracy_fake,
        "system_prompt": systemPrompt,
        "user_prompt": prompt,
        "one_shot": oneShot,
        #"imageBoost": imageBoost
    }
    if oneShot:
        results["one_shot_image"] = exampleImage
        results["tag_shot_image"] = labExample
    return results


def saveAllJson(metrics, responses, PromptITA, modelName, i):
    outputData = {
        "metrics": metrics,
        "responses": responses
    }
    Path("resultsJSON").mkdir(exist_ok=True)

    # Imposta lingua
    language_tag = "ITA" if PromptITA else "ENG"

    # Timestamp per identificare diversi tentativi
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    # Pulisci MODEL_NAME da caratteri non ammessi nei nomi file
    safe_model_name = modelName.replace(":", "_").replace("/", "_")
    
    # Costruisci filename
    filename = f"resultsJSON/real-vs-fake_{safe_model_name}_PromptType-{i}_{language_tag}_{timestamp}_result.json"
    # Salva JSON
    with open(filename, "w") as f:
        json.dump(outputData, f, indent=4)

    print(f"Results saved to {filename}.")

In [ ]:
# VALORI PER CARICARE IL DATASET
saveDatasetDirectory = False
startMiniDt = False
NAME = "oneshot/test_OS"
MAX_IMAGES = 300
SHUFFLE = True
# ======================================

# VALORI PER IL PROMPT
INDEX_PROMPT =  3 # (0-6)
IS_ITALIAN = False
SHOW_IMAGES = False
UNCERTAIN_EN = True
IMAGE_BOOST= False
# ===================================
# VALORI PER IL MODELLO

MODEL_NAME = "llava:7b"
# ===================================
# MODALITA' AUTOMATICA
AUTO_ON = True
# ===================================
# MODALITA' ONESHOT
ONESHOT = False
TAG_IMAGE = False
ONE_SHOT_IMAGE="/kaggle/input/oneshot/photoEx/fake/entniaman.jpg"
# MODALITA' FEWSHOT

FEWSHOT = False
FakeExample = "/kaggle/input/oneshot/photoEx/fake/perfectman.jpg"
RealExample = "/kaggle/input/oneshot/photoEx/real/man.jpg"

In [ ]:
if startMiniDt:
    images_with_labels, fakes, reals = loadDataset(MAX_IMAGES)
    saveDataset(images_with_labels, NAME)
else:
    images_with_labels, fakes, reals = loadExistingDataset(NAME)
    

if SHUFFLE:
    shuffleDataset(images_with_labels)

# prompt section
#oneShotMessage = None
if TAG_IMAGE:
    isFake = False
else:
    isFake = True
if AUTO_ON:
    for INDEX_PROMPT in range(7):  # Prompt da 0 a 6
        for IS_ITALIAN in [False, True]:  # Prima inglese, poi italiano
            userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
            systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)

            print(f"\n--- Prompt {INDEX_PROMPT} | Language: {'Italian' if IS_ITALIAN else 'English'} ---")
            print("Prompt:", userPrompt)

            # Inizializza le metriche
            counters = initMetrics()
            results = []

            # Ciclo principale di analisi immagini
            for img_path, label in tqdm(images_with_labels, desc=f"Analyzing images (Prompt {INDEX_PROMPT}, {'IT' if IS_ITALIAN else 'EN'})"):
                print(img_path)
                result, counters = analyze_image(
                    img_path, label, userPrompt, MODEL_NAME,
                    ONESHOT, systemPrompt,counters, SHOW_IMAGES, 
                    ONE_SHOT_IMAGE, isFake, IS_ITALIAN, FEWSHOT, RealExample, FakeExample
                )
                results.append(result)

            # Salvataggio dei risultati
            jsonINFO = analizeMetrics(
                counters, images_with_labels, userPrompt,
                systemPrompt, ONESHOT, ONE_SHOT_IMAGE, TAG_IMAGE,
                reals, fakes, IMAGE_BOOST, 
            )

            saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)

else:
    userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
    systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)
    
    print("you choose " + userPrompt)
    # initialize metrics
    counters = initMetrics()
    results = []
    # Main analysis loop
    for img_path, label in tqdm(images_with_labels, desc=" Analyzing images"):
        print(img_path)
        result, counters = analyze_image(
                    img_path, label, userPrompt, MODEL_NAME,
                    ONESHOT, systemPrompt,counters, SHOW_IMAGES, 
                    ONE_SHOT_IMAGE, isFake, IS_ITALIAN, FEWSHOT, RealExample, FakeExample
                )
        results.append(result)
    jsonINFO = analizeMetrics(
                counters, images_with_labels, userPrompt,
                systemPrompt, ONESHOT, ONE_SHOT_IMAGE, TAG_IMAGE,
                reals, fakes, IMAGE_BOOST, 
            )
    saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)